In [54]:
import os
os.chdir(r'C:\Users\Navid\Documents\ds_salary_proj')

In [55]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [56]:
df = pd.read_csv('eda_data_modified')

In [57]:
df['seniority'].value_counts()

0    705
1    254
Name: seniority, dtype: int64

In [58]:
# df = df.loc[df.Industry != '-1']
df = df.loc[df['Type of ownership'] != 'Unknown']

In [59]:
df = df.loc[(df.Size != '-1') & (df.Size != 'Unknown')]

In [60]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Job Title', 'Salary Range',
       'Job Description', 'Rating', 'Company Name', 'Location', 'Headquarters',
       'Size', 'Founded', 'Type of ownership', 'Industry', 'Sector', 'Revenue',
       'Competitors', 'salary_type', 'per_hour', 'Employer_reported',
       'avg_salary', 'state', 'desc_length', 'seniority', 'title', 'python',
       'r studio', 'ML', 'spark', 'deep_learning', 'aws', 'phd', 'undergrad',
       'master'],
      dtype='object')

In [61]:
df_final = df[['avg_salary' ,'Size','Type of ownership', 'Rating',
               'Sector', 'Revenue','per_hour', 'seniority', 'Employer_reported',
               'state', 'desc_length','aws', 'title', 'python', 'spark',
       'deep_learning','phd', 'master', 'Employer_reported', 'undergrad']]

In [62]:
df = df[df.Rating != -1]

In [63]:
#removing skewness
from scipy import stats
df['avg_salary'] ,fitted_lambda1 = stats.boxcox(df['avg_salary'])

In [64]:
df_dummy = pd.get_dummies(df_final)

In [65]:
# df_dummy.drop('Industry_other_industries',axis=1,inplace=True)

In [66]:
# train test split
from sklearn.model_selection import train_test_split
x = df_dummy.drop('avg_salary' , axis = 1)
y = df_dummy['avg_salary'].values
X_train , X_test , y_train , y_test = train_test_split(x , y , test_size = 0.2 , random_state = 1) 

In [69]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
lr = LinearRegression().fit(X_train,y_train)
cross_val_score(lr,X_train,y_train,scoring='neg_mean_absolute_error',cv=2).mean()

-14.164391538109065

In [70]:
import statsmodels.api as sm
X_sm = sm.add_constant(x)
model = sm.OLS(y,X_sm)
model.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.754
Model:                            OLS   Adj. R-squared:                  0.724
Method:                 Least Squares   F-statistic:                     25.36
Date:                Sat, 03 Oct 2020   Prob (F-statistic):          6.34e-198
Time:                        16:59:46   Log-Likelihood:                -4016.9
No. Observations:                 958   AIC:                             8242.
Df Residuals:                     854   BIC:                             8748.
Df Model:                         103                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
const                                               35.8157      3.653      9.805      0.000      28.646      42.985
Rating                                               5.4558      1.179      4.626      0.000       3.141       7.771
per_hour                                          9.346e-14   3.93e-14      2.381      0.017    1.64e-14    1.71e-13
seniority                                           22.5022      1.452     15.494      0.000      19.652      25.353
Employer_reported                                    2.5171      1.652      1.523      0.128      -0.726       5.760
desc_length                                          0.0005      0.000      0.977      0.329      -0.000       0.001
aws                                                  0.7784      1.495      0.521      0.603      -2.157       3.714
python                                               0.5947      1.561      0.381      0.703      -2.470       3.659
spark                                               -1.5401      1.803     -0.854      0.393      -5.079       1.998
deep_learning                                        2.7318      2.472      1.105      0.269      -2.120       7.584
phd                                                  2.0551      2.184      0.941      0.347      -2.231       6.342
master                                               0.8505      1.499      0.567      0.571      -2.093       3.794
Employer_reported                                    2.5171      1.652      1.523      0.128      -0.726       5.760
undergrad                                           -1.6258      1.398     -1.163      0.245      -4.370       1.119
Size_1 to 50 Employees                              -7.5834      3.687     -2.057      0.040     -14.820      -0.347
Size_10000+ Employees                                9.0967      2.576      3.531      0.000       4.041      14.153
Size_1001 to 5000 Employees                          8.5452      1.574      5.428      0.000       5.455      11.635
Size_201 to 500 Employees                            3.9790      1.723      2.309      0.021       0.597       7.361
Size_5001 to 10000 Employees                        13.2622      2.415      5.491      0.000       8.521      18.003
Size_501 to 1000 Employees                           4.0181      1.815      2.214      0.027       0.456       7.580
Size_51 to 200 Employees                             4.4980      2.004      2.244      0.025       0.564       8.432
Type of ownership_College / University               7.7535     12.763      0.607      0.544     -17.298      32.805
Type of ownership_Company - Private                  5.0302      3.078      1.634      0.103      -1.011      11.071
Type of ownership_Company - Public       

In [71]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
grid_param = {'n_estimators' : range(280,380,20) }
grid = GridSearchCV(RandomForestRegressor(n_jobs=-1 ,random_state = 0), param_grid = grid_param , cv =3).fit(X_train , y_train)



In [72]:
print(grid.best_score_,grid.best_estimator_)

0.8107305367274327 RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=300, n_jobs=-1, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)


In [73]:
df_dummy.corr()['avg_salary'].sort_values(ascending =False)[0:40]

avg_salary                                   1.000000
title_data scientist                         0.505901
python                                       0.388294
seniority                                    0.374309
state_CA                                     0.335552
phd                                          0.262069
deep_learning                                0.242833
spark                                        0.222591
Sector_Information Technology                0.188893
title_machine learning engineerr             0.181574
aws                                          0.178079
Sector_Biotech & Pharmaceuticals             0.153746
title_data engineer                          0.152202
state_MA                                     0.141062
Rating                                       0.139255
Size_51 to 200 Employees                     0.127443
Sector_Government                            0.101099
master                                       0.097109
desc_length                 

In [74]:
grid.best_estimator_.score(X_test,y_test)

0.8185043041027309